# Automated ML

In [45]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.run import Run
from azureml.train.automl import AutoMLConfig
from azureml.exceptions import ComputeTargetException
from azureml.widgets import RunDetails
import pandas as pd
import logging
import json
import joblib
import warnings
warnings.filterwarnings('ignore')

## Dataset

### Overview

"Airbnb for Boston with fraud detcetion" data was downloaded from Kaggle with the following link:

https://www.kaggle.com/datasets/hawkingcr/airbnb-for-boston-with-fraud-detection/download?datasetVersionNumber=1

The downloaded file was saved as "output.csv" in the "data" directory. The dataset aims to classify whether an Airbnb listing is a fraud or not.

A notebook file named "data_process.ipyng" was created to perform some pre-processing on the data. Firstly, a correlation analysis was conducted with the target column "fraud" to identify and remove some non-significant features. Next, the data was split into "train.csv" and "test.csv" sets, and the balance of the training data was examined. Due to the class imbalance in the training target, an upsampling technique was applied to address this imbalance.

A script file named "data_set.py" was created in the scripts directory. Two functions getTrainingDataset() and getTestDataset() are defined to register traing and test data as dataset in the workspace.

### Get Training Dataset

In [46]:
from scripts.data_set import getTrainingDataset

ws = Workspace.from_config()

experiment_name = 'udacity-aml-capstone-automl'
experiment=Experiment(ws, experiment_name)

train_ds = getTrainingDataset(ws)
train_ds.to_pandas_dataframe().head()


found existing traing dataset


,host_response_rate,host_identity_verified,host_total_listings_count,is_location_exact,property_type,accommodates,price,minimum_nights,number_of_reviews,review_scores_rating,instant_bookable,cancellation_policy,reviews_per_month,fraud
0,95,1,3,1,8,2,6500,2,8,93.00,0,1,0.63,1
1,100,1,1,1,0,8,50000,1,88,98.00,0,1,4.20,1
2,100,1,1,1,8,2,9000,1,192,95.00,0,1,5.58,1
3,90,1,1,1,0,2,11500,1,54,88.00,1,2,3.58,1
4,92,0,8,1,2,6,27500,2,29,91.00,1,2,0.72,1


### Create or Attach an AmlCompute cluster

In [47]:

cluster_name = "my-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_D2_V2", max_nodes=4
    )
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True)


Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

I picked the following configuration parameters:

iteration_timeout_minutes: Time limit in minutes for each iteration. Here it is set to 10 minutes for the relative small dataset.

experiment_timeout_hours: Maximum amount of time that all iterations combined can take before the experiment terminates. Here it is set to 30 minutes. Increase this value if needed.

enable_early_stopping: Flag to enable early termination if the score is not improving in the short term.

primary_metric: Since the original data is very imbalanced, "AUC_weighted" is chosen here.

featurization: By using auto, the experiment can preprocess the input data (handling missing data, converting text to numeric, etc.)

verbosity: Controls the level of logging.

n_cross_validation: Since validation data is not specified, Cross-Validation is adopted for training.

enable_onnx_compatible_models: After training, a ONNX model would be retrived.


In [48]:

automl_settings = {
    "iteration_timeout_minutes": 10,
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 4,
    "enable_early_stopping": True,
    "primary_metric": 'AUC_weighted',
    "featurization": 'auto',
    "verbosity": logging.INFO,
    "n_cross_validations": 5
}

automl_config = AutoMLConfig(
    task="classification",
    compute_target=compute_target,
    training_data=train_ds,
    label_column_name="fraud",
    enable_onnx_compatible_models=True,
    **automl_settings)

In [49]:
# Submit experiment
auto_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-aml-capstone-automl,AutoML_1e7ad3fd-249d-4f29-92fa-1a3d1c5a40dd,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details


In [50]:
RunDetails(auto_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model



In [51]:
# Retrieve best automl model.

auto_run.wait_for_completion(show_output=True)
assert(auto_run.get_status() == "Completed")

best_auto_run, best_model = auto_run.get_output()
# best_auto_child = auto_run.get_best_child()


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-aml-capstone-automl,AutoML_1e7ad3fd-249d-4f29-92fa-1a3d1c5a40dd,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

**********************************************************************************

Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

In [52]:
# print best run out files

print(best_auto_run.get_file_names())

best_auto_run_env = best_auto_run.get_environment()
print("best run_env: ", best_auto_run_env)
# print(best_auto_run.get_details())
# print("properties: ",best_auto_run.get_properties())


['accuracy_table', 'automl_driver.py', 'confusion_matrix', 'outputs/conda_env_v_1_0_0.yml', 'outputs/engineered_feature_names.json', 'outputs/env_dependencies.json', 'outputs/featurization_summary.json', 'outputs/generated_code/script.py', 'outputs/generated_code/script_run_notebook.ipynb', 'outputs/internal_cross_validated_models.pkl', 'outputs/model.onnx', 'outputs/model.pkl', 'outputs/model_onnx.json', 'outputs/pipeline_graph.json', 'outputs/run_id.txt', 'outputs/scoring_file_pbi_v_1_0_0.py', 'outputs/scoring_file_v_1_0_0.py', 'outputs/scoring_file_v_2_0_0.py']
best run_env:  Environment(Name: AzureML-AutoML,
Version: 142)


In [53]:
# Save the best model

output_dir = "./outputs"
if os.path.exists(output_dir) == False:
    os.mkdir(output_dir)
best_model_file = output_dir + "/best_automl_model.pkl"
joblib.dump(best_model,best_model_file)

print(best_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=False, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=True, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                     

In [54]:
# save run env

env_file = output_dir + "/automl_env.yml"
best_auto_run.download_file("outputs/conda_env_v_1_0_0.yml",env_file)


## Predict with best model

In [55]:
# prepare the test data frame

test_file = "./data/test.csv";
df_test = pd.read_csv(test_file)
df_test = df_test[pd.notnull(df_test['fraud'])]

y_test = df_test['fraud']
X_test = df_test.drop(['fraud'], axis=1)


In [56]:
from sklearn.metrics import confusion_matrix

ypred = best_model.predict(X_test)

# print confusion map with test dataset for best model
cm = confusion_matrix(y_test, ypred)
pd.DataFrame(cm).style.background_gradient(cmap='Blues', low=0, high=0.9)


,0,1
0,650,48
1,79,120


### Retrieve and save ONNX Model

In [57]:
from azureml.automl.runtime.onnx_convert import OnnxConverter

bestrun, onnx_mdl = auto_run.get_output(return_onnx_model=True)

onnx_fl_path = output_dir+"/best_model.onnx"
OnnxConverter.save_onnx_model(onnx_mdl, onnx_fl_path)


### Predict with ONNX model


In [58]:
import sys
# import json
from azureml.automl.core.onnx_convert import OnnxConvertConstants
from azureml.train.automl import constants
from azureml.automl.runtime.onnx_convert import OnnxInferenceHelper

print(constants.MODEL_RESOURCE_PATH_ONNX)

def get_onnx_res(run):
    res_path = output_dir + "/onnx_resource.json"
    run.download_file(
        name=constants.MODEL_RESOURCE_PATH_ONNX, output_file_path=res_path
    )
    with open(res_path) as f:
        result = json.load(f)
    return result


if sys.version_info < OnnxConvertConstants.OnnxIncompatiblePythonVersion:
    # test_df = test_dataset.to_pandas_dataframe()
    mdl_bytes = onnx_mdl.SerializeToString()
    onnx_result = get_onnx_res(bestrun)

    onnxrt_helper = OnnxInferenceHelper(mdl_bytes, onnx_result)
    pred_onnx, pred_prob_onnx = onnxrt_helper.predict(X_test)
    #print(pred_onnx)
    #print(pred_prob_onnx)
else:
    print("Please use Python version 3.6 or higher to run the inference helper.")
    

outputs/model_onnx.json


2023-08-15 22:21:46.841165828 [W:onnxruntime:, execution_frame.cc:806 VerifyOutputSizes] Expected shape from model of {1} does not match actual shape of {897} for output output_label


In [59]:
# print confusion map with test dataset for ONNX model

cm = confusion_matrix(y_test, pred_onnx)
pd.DataFrame(cm).style.background_gradient(cmap='Blues', low=0, high=0.9)

,0,1
0,647,51
1,79,120


## Model Registration


In [60]:

model_name = best_auto_run.properties["model_name"]
#print(model_name)
description = "AutoML Model to predict Airbnb fraud listing"
tags = None
registered_model = auto_run.register_model(
    model_name=model_name, description=description, tags=tags
)

print(
    auto_run.model_id
) 

print("registered model: ",registered_model)

AutoML1e7ad3fd214
registered model:  Model(workspace=Workspace.create(name='quick-starts-ws-240304', subscription_id='1b944a9b-fdae-4f97-aeb1-b7eea0beac53', resource_group='aml-quickstarts-240304'), name=AutoML1e7ad3fd214, id=AutoML1e7ad3fd214:1, version=1, tags={}, properties={})


### Show registered model

In [ ]:
from azureml.core.model import Model

# Retrieve registered model
model_registered = Model(ws, model_name)

print(model_registered)

## Deploy the model in ACI

In [61]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
# from azureml.core.model import Model
from azureml.core.environment import Environment

# download score file

script_file_name = output_dir + "/score.py"
best_auto_run.download_file("outputs/scoring_file_v_2_0_0.py", script_file_name)

inference_config = InferenceConfig(
    environment=best_auto_run.get_environment(), entry_script=script_file_name
)

aciconfig = AciWebservice.deploy_configuration(
    cpu_cores=2,
    memory_gb=2,
    tags={"type": "automl_classification"},
    description="Automl Classification Service",
)

aci_service_name = model_name.lower()
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [registered_model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
# print("endpoint uri: ",aci_service.scoring_uri)
print("endpoint state: ",aci_service.state)

automl1e7ad3fd214
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-08-15 22:21:52+00:00 Creating Container Registry if not exists.
2023-08-15 22:21:52+00:00 Registering the environment.
2023-08-15 22:21:53+00:00 Use the existing image.
2023-08-15 22:21:54+00:00 Submitting deployment to compute.
2023-08-15 22:22:01+00:00 Checking the status of deployment automl1e7ad3fd214..
2023-08-15 22:24:25+00:00 Checking the status of inference endpoint automl1e7ad3fd214.
Succeeded
ACI service creation operation finished, operation "Succeeded"
endpoint state:  Healthy


### Request to web service

The following is an example endpoint request

In [72]:
import urllib.request

data =  {
  "Inputs": {
    "data": [
      {
        "host_response_rate": 100,
        "host_identity_verified": 0,
        "host_total_listings_count": 1,
        "is_location_exact": 1,
        "property_type": 0,
        "accommodates": 2,
        "price": 5000,
        "minimum_nights": 15,
        "number_of_reviews": 0,
        "review_scores_rating": 100.0,
        "instant_bookable": 1,
        "cancellation_policy": 2,
        "reviews_per_month": 1.0
      }
    ]
  },
  "GlobalParameters": {
    "method": "predict"
  }
}

body = str.encode(json.dumps(data))

# url = aci_service.scoring_uri
url = "http://cefae4be-6f92-4f49-b006-17c1aef41e0e.westeurope.azurecontainer.io/score"
#print("endpoint url:", url)

headers = {'Content-Type':'application/json'}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'{"Results": [0]}'


### Print the logs of the web service

In [63]:
aci_service.get_logs()

'2023-08-15T22:23:56,951711500+00:00 - gunicorn/run \n2023-08-15T22:23:56,946304400+00:00 - rsyslog/run \n2023-08-15T22:23:56,958465200+00:00 | gunicorn/run | \n2023-08-15T22:23:56,966284500+00:00 | gunicorn/run | ###############################################\n2023-08-15T22:23:56,971133500+00:00 | gunicorn/run | AzureML Container Runtime Information\n2023-08-15T22:23:56,974500300+00:00 | gunicorn/run | ###############################################\n2023-08-15T22:23:56,977807600+00:00 - nginx/run \n2023-08-15T22:23:56,978863600+00:00 | gunicorn/run | \n2023-08-15T22:23:56,989174000+00:00 | gunicorn/run | \n2023-08-15T22:23:57,000323200+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230628.v2\n2023-08-15T22:23:57,010430000+00:00 | gunicorn/run | \n2023-08-15T22:23:57,018178300+00:00 | gunicorn/run | \n2023-08-15T22:23:57,023085500+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/

## Cleanup Resources

In [64]:
aci_service.delete()

compute_target.delete()
